# Market/Property Evaluation Tool for Residential Real Estate Investors

Our project is designed to provide residential real estate investors with a tool that uses calculated metrics to help narrow their search for investment properties based on a set of criteria.  These metrics can be both market (city) and property based so that investors can start by narrowing from a list of cities, then filter individual properties within that market to determine which rental houses show attractive investment ratios.

**Questions our tool will answer**
1. *"In which US market should I buy a rental property?"*
2. *"Which individual properties with in a market should I take a closer look at?"*

## Core Metrics

### We will use a set of core metrics to evaluate and filter data on a market and property level. A list of these metrics can be found below, along with their calculation and goal outcome.

**Market Level**

1. Price to Rent Ratio: Median Home Price / Annual Median Rent, *Goal: <15*

2. Annual Population Growth % (last 5 years)

3. Annual Job Growth % (last 5 years)

4. Quality of Life Index

5. Supply vs Demand:
    - Supply > Demand = "Buyer's Market"
    - Demand > Supply = "Seller's Market"

**Property Level**

1. Rent Ratio: Monthly Rent Estimate / Total Cost, *Goal: >1%*

2. Cap Rate:  Net Operating Income (NOI) / Total Cost, *Goal: >7%*

3. OpEx Ratio:  Operating Expenses/Gross Income, *Goal: <70%*

4. Price vs Comparables:  (List Price - Comp Price) / Price *Goal: >10%*


## Equations and Variables

### Other Formulas

1.  Total Cost = List Price + Closing Fees
2.  NOI = Gross Income - Annual Operating Expenses (OpEx)
3.  OpEx = Repairs/Maintenance + Annual Property Tax + Vacancy Rate + Insurance Estimate + Misc Expenses

### Based on the above calculations, we will need to gather the following individual data points (variable and potential source listed below):

1.  Address/List Price (Zillow API)
2.  Closing Fees (Formula: List Price * 3%)
3.  Gross Rent (Zillow API: "Rent Zestimate" * 12)
4.  Annual Property Tax (Zillow API)
5.  Insurance Estimate (Formula: (List Price/100) * $3.50
6.  Vacancy Rate (Formula: Gross Rent * 7%)
7.  Repairs/Maintenance (Formula: Gross Rent * 10%)
8.  Misc Expenses (Formula: Gross Rent * 5%)
9.  Median Home Price (Zillow csv: "Zillow Home Value Index")
10. Annual Median Rent (Zillow csv: "Zillow Observed Rent Index")
11. City Population Growth (? API)
12. City Job Growth (? API)
13. City Housing Supply (Zillow API)
14. City Housing Demand (Zillow API)
15. Quality of Life Index (Numbeo?)

## Output

The results for each individual property would show the following:

Address
List Price
Total Cost
Gross Rent
Rent Ratio
Cap Rate
OpEx Ratio
Price vs Comp




